# Feature Importance with Monte Carlo Cross-Validation

**Purpose:** Calculate scaled feature importance using CatBoost and Random Forest  
**Method:** Normalized feature importance scaled by MC-CV Recall scores  
**Based on:** [R Example](https://github.com/Jerome3590/phts/blob/main/graft-loss/feature_importance/replicate_20_features_MC_CV.R)  
**Updated:** November 2025  
**Hardware:** Optimized for EC2 (32 cores, 1TB RAM)  
**Validation:** Proper evaluation on unseen test data

## Key Features

✅ **Monte Carlo Cross-Validation** – up to 1000 random train/test splits (100-split runs used for faster iteration)  
✅ **Stratified Sampling** - Maintains target distribution  
✅ **Parallel Processing** - Fast execution with furrr/future (≈30 workers)  
✅ **95% Confidence Intervals** - Narrow, precise estimates (tighter with more splits)  
✅ **Multiple Models** - CatBoost (R) and Random Forest (R)  

## Methodology

This notebook implements the feature selection methodology:

1. Load cohort data from parquet files (same as FP-Growth notebook)
2. Create patient-level features (one-hot encoding of items)
3. For each model type:
   - Create 100–1000 stratified train/test splits
   - Train model on training set
   - Evaluate Recall on unseen test set
   - Extract feature importance
   - Aggregate results across splits
4. Normalize and scale feature importance by MC-CV Recall
5. Aggregate across models
6. Extract top features

## Expected Runtime

- **100 splits (current default):**
  - Local (4 cores): ~2–4 hours
  - Workstation (16 cores): ~1–2 hours
  - EC2 (32 cores, 1TB RAM): ~1–2 hours ✅ **RECOMMENDED FOR DEVELOPMENT**
- **1000 splits (extended / publication-level):**
  - Local (4 cores): 8–12+ hours
  - Workstation (16 cores): ~8–16 hours
  - EC2 (32 cores, 1TB RAM): ~10–20 hours ✅ **RECOMMENDED FOR FINAL RESULTS**


## 1. Setup and Configuration

Load required packages and configure parallel processing.


In [ ]:
# Check R version
R.version.string

# Load required packages
suppressPackageStartupMessages({
  library(here)
  library(dplyr)
  library(readr)
  library(tidyr)
  library(tibble)
  library(purrr)
  library(catboost)
  library(randomForest)
  library(rsample)    # For MC-CV
  library(furrr)      # For parallel processing
  library(future)     # For parallel backend
  library(progressr)  # For progress bars
  library(duckdb)     # For loading parquet files
  library(DBI)        # Database interface for DuckDB
})

cat("✓ All packages loaded successfully\n")


In [ ]:
# ============================================================
# DEBUG/TEST MODE - Quick testing before full run
# ============================================================
# Set DEBUG_MODE = TRUE for quick testing (5 splits, ~2-5 min)
# Set DEBUG_MODE = FALSE for full analysis (100 splits, ~1-2 hours on EC2)

DEBUG_MODE <- TRUE  # Change to TRUE for quick test

if (DEBUG_MODE) {
  cat("\n")
  cat("╔════════════════════════════════════════════════════════════════╗\n")
  cat("║                    🔍 DEBUG MODE ENABLED                       ║\n")
  cat("╚════════════════════════════════════════════════════════════════╝\n")
  cat("\n")
  cat("Quick test configuration:\n")
  cat("  • MC-CV Splits: 5 (instead of 100)\n")
  cat("  • Expected time: 2-5 minutes\n")
  cat("  • Purpose: Verify everything works before full run\n")
  cat("\n")
  cat("To run full analysis, set DEBUG_MODE = FALSE\n")
  cat("\n")
}

# Configuration
# NOTE: Target definition differs by cohort (already baked into cohort data via is_target_case):
#   - "opioid_ed": Target cases = patients with F1120/opioid ICD codes (any of 10 ICD columns)
#   - "non_opioid_ed": Target cases = patients with HCG ED visits (P51/O11/P33) WITHOUT opioid codes
# Controls (is_target_case=0) are sampled to maintain 5:1 ratio in both cohorts
COHORT_NAME <- "opioid_ed"  # Options: "opioid_ed" or "non_opioid_ed"
AGE_BAND <- "25-44"         # Change as needed
EVENT_YEAR <- 2016          # Change as needed

N_SPLITS <- if (DEBUG_MODE) 5 else 100  # MC-CV splits (5 for debug, 100 for development, 1000 for production)
TEST_SIZE <- 0.2             # Test set proportion (20%)
TRAIN_PROP <- 1 - TEST_SIZE  # Training proportion (80%)

# Scaling metric for feature importance
# Options: "recall" (default) or "logloss"
# - Recall: Higher is better (0-1), good for imbalanced classes, focuses on finding positives
# - LogLoss: Lower is better, measures probability calibration, penalizes overconfident errors
#   (will be inverted: 1/logloss for scaling, so higher = better)
SCALING_METRIC <- "recall"  # Change to "logloss" if preferred

# Model parameters
MODEL_PARAMS <- list(
  catboost = list(
    iterations = 100,
    learning_rate = 0.1,
    depth = 6,
    verbose = 0L,  # Turn off CatBoost logging (0L = integer 0)
    random_seed = 42
  ),
  random_forest = list(
    ntree = 100,
    mtry = NULL,  # Will be set to sqrt(n_features)
    nodesize = 1,
    maxnodes = NULL
  )
)

# Set up parallel processing
# EC2 optimization: Use 30 out of 32 cores (leave 2 for system)
N_WORKERS <- as.integer(Sys.getenv("N_WORKERS", "0"))
if (N_WORKERS < 1) {
  # Auto-detect: use all cores minus 2 for system
  total_cores <- parallel::detectCores()
  N_WORKERS <- max(1, total_cores - 2)
  cat(sprintf("Auto-detected %d cores, using %d workers\n", total_cores, N_WORKERS))
}
cat(sprintf("Setting up parallel processing with %d workers...\n", N_WORKERS))

# Increase future.globals.maxSize for large MC-CV splits object
# With 1TB RAM on EC2, we can handle large transfers
# Note: MC-CV splits and data matrices can be very large (30+ GB)
# Increase limit to accommodate large feature matrices (19,586 features × 31,146 patients)
options(future.globals.maxSize = 97 * 1024^3)  # 97 GB limit (increased from 20 GB)
cat("Set future.globals.maxSize to 97 GB\n")

plan(multisession, workers = N_WORKERS)

# Output directory
output_dir <- here("outputs")
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

cat("Output directory:", output_dir, "\n")
cat(sprintf("MC-CV Configuration: %d splits, %.0f/%.0f train/test split\n", 
            N_SPLITS, TRAIN_PROP * 100, TEST_SIZE * 100))


## 2. Data Loading

Load cohort data from parquet files (same logic as FP-Growth notebook).


In [ ]:
# Determine local data path (same as FP-Growth)
LOCAL_DATA_PATH <- Sys.getenv("LOCAL_DATA_PATH", "/mnt/nvme/cohorts")
if (!dir.exists(LOCAL_DATA_PATH)) {
  # Try Windows path
  LOCAL_DATA_PATH <- Sys.getenv("LOCAL_DATA_PATH", "C:/Projects/pgx-analysis/data/gold/cohorts_F1120")
}

parquet_file <- file.path(LOCAL_DATA_PATH, 
                          paste0("cohort_name=", COHORT_NAME),
                          paste0("event_year=", EVENT_YEAR),
                          paste0("age_band=", AGE_BAND),
                          "cohort.parquet")

if (!file.exists(parquet_file)) {
  stop(sprintf("Cohort file not found: %s\nPlease check LOCAL_DATA_PATH and file structure.", parquet_file))
}

cat(sprintf("Loading from: %s\n", parquet_file))

# Load using DuckDB
con <- dbConnect(duckdb::duckdb(), dbdir = ":memory:")

# Load cohort data (same columns as FP-Growth notebook)
# NOTE: Use is_target_case as target variable (target column is hardcoded to 1 in cohort SQL)
# is_target_case definition depends on COHORT_NAME:
#   - opioid_ed: 1 = patients with opioid ICD codes (F1120, etc.), 0 = controls
#   - non_opioid_ed: 1 = patients with HCG ED visits (no opioid codes), 0 = controls
query <- sprintf("
  SELECT 
    mi_person_key,
    is_target_case as target,  -- Use is_target_case: 1=target case, 0=control
    drug_name,
    primary_icd_diagnosis_code,
    two_icd_diagnosis_code,
    three_icd_diagnosis_code,
    four_icd_diagnosis_code,
    five_icd_diagnosis_code,
    procedure_code,
    event_type
  FROM read_parquet('%s')
", parquet_file)

cohort_data <- dbGetQuery(con, query)
dbDisconnect(con)

cat(sprintf("Loaded %d event-level records\n", nrow(cohort_data)))
cat(sprintf("Unique patients: %d\n", length(unique(cohort_data$mi_person_key))))

# Print event type distribution
cat("\nEvent type distribution:\n")
event_dist <- cohort_data %>%
  count(event_type) %>%
  mutate(pct = 100 * n / sum(n))
for (i in 1:nrow(event_dist)) {
  cat(sprintf("  %s: %d (%.1f%%)\n", event_dist$event_type[i], event_dist$n[i], event_dist$pct[i]))
}

# Print target distribution (at event level)
cat("\nTarget distribution (event-level):\n")
target_dist <- cohort_data %>%
  count(target) %>%
  mutate(pct = 100 * n / sum(n))
for (i in 1:nrow(target_dist)) {
  cat(sprintf("  Target %d: %d events (%.1f%%)\n", target_dist$target[i], target_dist$n[i], target_dist$pct[i]))
}

# Print target distribution (patient-level)
cat("\nTarget distribution (patient-level):\n")
patient_target_dist <- cohort_data %>%
  select(mi_person_key, target) %>%
  distinct() %>%
  count(target) %>%
  mutate(pct = 100 * n / sum(n))
for (i in 1:nrow(patient_target_dist)) {
  if (is.na(patient_target_dist$target[i])) {
    cat(sprintf("  Target NA: %d patients (%.1f%%) ⚠️  ISSUE: NULL is_target_case in source data\n", 
                patient_target_dist$n[i], patient_target_dist$pct[i]))
  } else {
    cat(sprintf("  Target %d: %d patients (%.1f%%)\n", 
                patient_target_dist$target[i], patient_target_dist$n[i], patient_target_dist$pct[i]))
  }
}

# Check for patients with inconsistent target values across events
cat("\nChecking for patients with inconsistent target values:\n")
inconsistent_patients <- cohort_data %>%
  select(mi_person_key, target) %>%
  distinct() %>%
  group_by(mi_person_key) %>%
  summarise(n_unique_targets = n_distinct(target, na.rm = FALSE), .groups = 'drop') %>%
  filter(n_unique_targets > 1)

if (nrow(inconsistent_patients) > 0) {
  cat(sprintf("  ⚠️  WARNING: %d patients have inconsistent target values across events!\n", 
              nrow(inconsistent_patients)))
  cat("  This suggests is_target_case is calculated per-event instead of per-patient.\n")
  cat("  Using first() value per patient as workaround.\n")
} else {
  cat("  ✓ No inconsistencies found\n")
}


## 3. Feature Engineering

Create patient-level features with categorical factor columns for CatBoost.

**Approach:**
- Create feature columns where each column represents an item (drug, ICD code, CPT code)
- Each column is a factor with levels based on the actual categorical values (item names)
- Target remains binary (0/1)
- For Random Forest: use numeric binary (0/1) format


In [ ]:
# Extract all unique items per patient
cat("\nCreating patient-level features...\n")

patient_items <- cohort_data %>%
  # Drug names (pharmacy events)
  filter(!is.na(drug_name) & drug_name != "" & event_type == "pharmacy") %>%
  select(mi_person_key, item = drug_name) %>%
  # ICD codes (medical events) - all 5 columns
  bind_rows(
    cohort_data %>%
      filter(event_type == "medical") %>%
      select(mi_person_key, item = primary_icd_diagnosis_code) %>%
      filter(!is.na(item) & item != ""),
    cohort_data %>%
      filter(event_type == "medical") %>%
      select(mi_person_key, item = two_icd_diagnosis_code) %>%
      filter(!is.na(item) & item != ""),
    cohort_data %>%
      filter(event_type == "medical") %>%
      select(mi_person_key, item = three_icd_diagnosis_code) %>%
      filter(!is.na(item) & item != ""),
    cohort_data %>%
      filter(event_type == "medical") %>%
      select(mi_person_key, item = four_icd_diagnosis_code) %>%
      filter(!is.na(item) & item != ""),
    cohort_data %>%
      filter(event_type == "medical") %>%
      select(mi_person_key, item = five_icd_diagnosis_code) %>%
      filter(!is.na(item) & item != "")
  ) %>%
  # CPT codes (medical events)
  bind_rows(
    cohort_data %>%
      filter(!is.na(procedure_code) & procedure_code != "" & event_type == "medical") %>%
      select(mi_person_key, item = procedure_code)
  ) %>%
  distinct() %>%
  filter(!is.na(item) & item != "")

cat(sprintf("Extracted %d unique patient-item pairs\n", nrow(patient_items)))
cat(sprintf("Unique items: %d\n", length(unique(patient_items$item))))

# Get target per patient
patient_targets <- cohort_data %>%
  select(mi_person_key, target) %>%
  distinct() %>%
  group_by(mi_person_key) %>%
  summarise(target = first(target), .groups = 'drop')

# Create feature matrix (one column per item)
# For CatBoost: Use actual item names as categorical values
# For Random Forest: Use binary 0/1

cat("\nCreating feature matrix...\n")

# Get all unique items to create columns
all_unique_items <- sort(unique(patient_items$item))
cat(sprintf("Creating %d feature columns (one per unique item)\n", length(all_unique_items)))

# FOR CATBOOST: Create feature matrix where each column represents an item
# Value is the item name itself (categorical), or NA if patient doesn't have it
cat("\nCreating CatBoost format (item names as categorical values)...\n")
feature_matrix_catboost <- patient_items %>%
  pivot_wider(
    id_cols = mi_person_key,
    names_from = item,
    values_from = item,  # Use item name itself as value (not 0/1)
    values_fill = NA_character_,  # NA if patient doesn't have the item
    names_prefix = "item_"
  ) %>%
  left_join(patient_targets, by = "mi_person_key")

# Validate join - check if target column has values
cat("\nValidating feature matrix join:\n")
cat(sprintf("  feature_matrix_catboost rows: %d\n", nrow(feature_matrix_catboost)))
cat(sprintf("  Target column present: %s\n", "target" %in% names(feature_matrix_catboost)))
if ("target" %in% names(feature_matrix_catboost)) {
  cat(sprintf("  Target NAs: %d (%.1f%%)\n", 
              sum(is.na(feature_matrix_catboost$target)), 
              100 * mean(is.na(feature_matrix_catboost$target))))
  cat(sprintf("  Target positives: %d (%.1f%%)\n", 
              sum(feature_matrix_catboost$target == 1, na.rm = TRUE),
              100 * mean(feature_matrix_catboost$target == 1, na.rm = TRUE)))
  cat(sprintf("  Target negatives: %d (%.1f%%)\n", 
              sum(feature_matrix_catboost$target == 0, na.rm = TRUE),
              100 * mean(feature_matrix_catboost$target == 0, na.rm = TRUE)))
  
  # Check for patients missing from patient_targets
  missing_targets <- feature_matrix_catboost %>%
    filter(is.na(target)) %>%
    select(mi_person_key) %>%
    distinct()
  if (nrow(missing_targets) > 0) {
    cat(sprintf("  WARNING: %d patients missing target values\n", nrow(missing_targets)))
  }
}

# FOR CATBOOST: Convert ALL feature columns to factors with actual item names as levels
cat("\nCreating CatBoost format (categorical factors with item names as levels)...\n")
data_catboost <- feature_matrix_catboost %>%
  select(-mi_person_key) %>%
  # Convert ALL feature columns (except target) to factors
  # Each column has levels: NA (patient doesn't have item) or the item name itself
  mutate(across(-target, ~ as.factor(.x)))

y_catboost <- data_catboost$target
X_catboost <- data_catboost %>% select(-target)

cat(sprintf("CatBoost format:\n"))
cat(sprintf("  Patients: %d\n", nrow(X_catboost)))
cat(sprintf("  Features: %d (categorical factors with item names as levels)\n", ncol(X_catboost)))
cat(sprintf("  Target distribution: %d (%.1f%%) positive, %d (%.1f%%) negative\n",
            sum(y_catboost == 1), 100 * mean(y_catboost == 1), 
            sum(y_catboost == 0), 100 * mean(y_catboost == 0)))

# FOR RANDOM FOREST: Create binary feature matrix (0/1)
cat("\nCreating Random Forest format (numeric binary 0/1)...\n")
feature_matrix_rf <- patient_items %>%
  mutate(value = 1) %>%
  pivot_wider(
    id_cols = mi_person_key,
    names_from = item,
    values_from = value,
    values_fill = 0,
    names_prefix = "item_"
  ) %>%
  left_join(patient_targets, by = "mi_person_key")

data_rf <- feature_matrix_rf %>%
  select(-mi_person_key)
# Keep as numeric (0/1) for Random Forest

y_rf <- data_rf$target
X_rf <- data_rf %>% select(-target)

cat(sprintf("Random Forest format:\n"))
cat(sprintf("  Patients: %d\n", nrow(X_rf)))
cat(sprintf("  Features: %d (numeric binary)\n", ncol(X_rf)))

# Use CatBoost format by default (categorical factors)
data <- data_catboost
y <- y_catboost
X <- X_catboost


## 4. Helper Functions

Define functions for training models and calculating feature importance.


In [ ]:
# Train CatBoost model (R) - uses categorical factor features
train_catboost_r <- function(X_train, y_train, params) {
  # X_train should have ALL columns as factors (categorical features)
  # R's CatBoost automatically detects factor columns as categorical - no need to specify
  # Verify all columns are factors
  factor_cols <- names(X_train)[sapply(X_train, is.factor)]
  if (length(factor_cols) != ncol(X_train)) {
    warning(sprintf("Not all columns are factors! Factors: %d, Total: %d", 
                    length(factor_cols), ncol(X_train)))
  }
  
  # Create pool - R CatBoost automatically handles factor columns as categorical
  train_pool <- catboost.load_pool(
    data = X_train, 
    label = y_train
  )
  
  catboost_params <- list(
    iterations = params$iterations,
    learning_rate = params$learning_rate,
    depth = params$depth,
    loss_function = 'Logloss',
    eval_metric = 'Recall',
    verbose = params$verbose,
    logging_level = 'Silent',  # Suppress CatBoost logging to avoid thread safety warnings
    random_seed = params$random_seed
  )
  
  model <- catboost.train(train_pool, NULL, catboost_params)
  return(model)
}

# Train Random Forest model (R)
train_random_forest_r <- function(X_train, y_train, params) {
  if (is.null(params$mtry)) {
    params$mtry <- floor(sqrt(ncol(X_train)))
  }
  
  y_train_factor <- as.factor(y_train)
  
  model <- randomForest(
    x = X_train,
    y = y_train_factor,
    ntree = params$ntree,
    mtry = params$mtry,
    nodesize = params$nodesize,
    maxnodes = params$maxnodes,
    importance = TRUE
  )
  
  return(model)
}

# Get feature importance from CatBoost (R)
get_importance_catboost_r <- function(model, X_test) {
  # R's CatBoost automatically detects factor columns as categorical - no need to specify
  # Verify all columns are factors (should match training)
  factor_cols <- names(X_test)[sapply(X_test, is.factor)]
  if (length(factor_cols) != ncol(X_test)) {
    warning(sprintf("Test data: Not all columns are factors! Factors: %d, Total: %d", 
                    length(factor_cols), ncol(X_test)))
  }
  
  # Create test pool - R CatBoost automatically handles factor columns as categorical
  test_pool <- catboost.load_pool(data = X_test)
  
  # Get feature importance - ensure it returns a vector
  importance <- catboost.get_feature_importance(model, pool = test_pool, type = 'PredictionValuesChange')
  
  # Ensure importance is a named vector with correct length
  if (length(importance) == 1 && ncol(X_test) > 1) {
    warning(sprintf("Feature importance returned single value instead of vector (expected %d features)", ncol(X_test)))
    # Fallback: return zeros with feature names
    importance <- setNames(rep(0, ncol(X_test)), names(X_test))
  } else if (length(importance) != ncol(X_test)) {
    warning(sprintf("Feature importance length mismatch: got %d, expected %d", length(importance), ncol(X_test)))
    # Try to pad or truncate to match
    if (length(importance) < ncol(X_test)) {
      importance <- c(importance, rep(0, ncol(X_test) - length(importance)))
    } else {
      importance <- importance[1:ncol(X_test)]
    }
    names(importance) <- names(X_test)
  } else if (is.null(names(importance))) {
    # Ensure names are set
    names(importance) <- names(X_test)
  }
  
  return(importance)
}

# Get feature importance from Random Forest (R)
get_importance_random_forest_r <- function(model) {
  importance <- importance(model)[, "MeanDecreaseGini"]
  return(importance)
}

# Calculate metrics for scaling
# Handle NA values: remove them before calculation

# Calculate Recall
calculate_recall <- function(y_true, y_pred) {
  # Check inputs
  if (length(y_true) != length(y_pred)) {
    warning(sprintf("Length mismatch: y_true=%d, y_pred=%d", length(y_true), length(y_pred)))
    return(0)
  }
  
  # Remove NA values from both vectors
  valid_idx <- !is.na(y_true) & !is.na(y_pred)
  
  if (sum(valid_idx) == 0) {
    warning(sprintf("No valid predictions for recall calculation (y_true: %d NAs/%d, y_pred: %d NAs/%d)", 
                    sum(is.na(y_true)), length(y_true), sum(is.na(y_pred)), length(y_pred)))
    return(0)
  }
  
  y_true_clean <- y_true[valid_idx]
  y_pred_clean <- y_pred[valid_idx]
  
  # Check if we have any valid data
  if (length(y_true_clean) == 0) {
    warning("No valid data after filtering NAs")
    return(0)
  }
  
  tp <- sum((y_true_clean == 1) & (y_pred_clean == 1))
  fn <- sum((y_true_clean == 1) & (y_pred_clean == 0))
  
  # Handle edge case: no positive cases in true labels
  if (tp + fn == 0) {
    warning(sprintf("No positive cases in y_true for recall calculation (total valid: %d, positives: %d)", 
                    length(y_true_clean), sum(y_true_clean == 1)))
    return(0)
  }
  
  return(tp / (tp + fn))
}

# Calculate LogLoss (logarithmic loss)
# Lower is better, so we'll invert it for scaling (1/logloss or use negative)
calculate_logloss <- function(y_true, y_pred_proba) {
  # Remove NA values
  valid_idx <- !is.na(y_true) & !is.na(y_pred_proba)
  if (sum(valid_idx) == 0) {
    warning("No valid predictions for logloss calculation")
    return(Inf)  # Return Inf so 1/logloss = 0
  }
  
  y_true_clean <- y_true[valid_idx]
  y_pred_proba_clean <- y_pred_proba[valid_idx]
  
  # Clip probabilities to avoid log(0) or log(1)
  y_pred_proba_clean <- pmax(pmin(y_pred_proba_clean, 1 - 1e-15), 1e-15)
  
  # Calculate logloss
  logloss <- -mean(y_true_clean * log(y_pred_proba_clean) + (1 - y_true_clean) * log(1 - y_pred_proba_clean))
  
  return(logloss)
}

# Predict with CatBoost (R) - returns binary predictions
predict_catboost_r <- function(model, X_test) {
  # R's CatBoost automatically detects factor columns as categorical - no need to specify
  # Create test pool - R CatBoost automatically handles factor columns as categorical
  test_pool <- catboost.load_pool(data = X_test)
  pred_proba <- catboost.predict(model, test_pool, prediction_type = 'Probability')
  
  # Handle NA values: if pred_proba is NA, default to 0 (negative class)
  pred <- ifelse(is.na(pred_proba), 0, ifelse(pred_proba > 0.5, 1, 0))
  
  # Ensure no NA values remain
  if (any(is.na(pred))) {
    warning("NA values in CatBoost predictions, replacing with 0")
    pred[is.na(pred)] <- 0
  }
  
  return(pred)
}

# Predict probabilities with CatBoost (R) - returns probability values
predict_proba_catboost_r <- function(model, X_test) {
  test_pool <- catboost.load_pool(data = X_test)
  pred_proba <- catboost.predict(model, test_pool, prediction_type = 'Probability')
  
  # Handle NA values
  if (any(is.na(pred_proba))) {
    warning("NA values in CatBoost probability predictions, replacing with 0.5")
    pred_proba[is.na(pred_proba)] <- 0.5
  }
  
  return(pred_proba)
}

# Predict with Random Forest (R) - returns binary predictions
predict_random_forest_r <- function(model, X_test) {
  pred <- predict(model, X_test, type = 'response')
  pred <- as.integer(pred) - 1  # Convert factor to 0/1
  
  # Handle NA values: if prediction is NA, default to 0 (negative class)
  if (any(is.na(pred))) {
    warning("NA values in Random Forest predictions, replacing with 0")
    pred[is.na(pred)] <- 0
  }
  
  return(pred)
}

# Predict probabilities with Random Forest (R) - returns probability values
predict_proba_random_forest_r <- function(model, X_test) {
  pred_proba <- predict(model, X_test, type = 'prob')[, 2]  # Get probability of class 1
  
  # Handle NA values
  if (any(is.na(pred_proba))) {
    warning("NA values in Random Forest probability predictions, replacing with 0.5")
    pred_proba[is.na(pred_proba)] <- 0.5
  }
  
  return(pred_proba)
}

cat("✓ Helper functions defined\n")


## 5. Monte Carlo Cross-Validation

Run MC-CV for each model type.


In [ ]:
# Create MC-CV splits (stratified by target)
# Note: We use the CatBoost format (categorical) for splits, but Random Forest will use its own format
cat("\n========================================\n")
cat("Creating MC-CV Splits\n")
cat("========================================\n")

# Validate data before creating splits
if (!"target" %in% names(data)) {
  stop("Target column not found in data. Cannot create MC-CV splits.")
}
if (all(is.na(data$target))) {
  stop("Target column is all NA. Cannot create MC-CV splits. Check data loading.")
}
cat(sprintf("Data for splits (before cleaning): nrow=%d, target NAs=%d, target positives=%d\n",
            nrow(data), sum(is.na(data$target)), sum(data$target == 1, na.rm = TRUE)))

# CRITICAL FIX: Remove any rows with NA target BEFORE creating splits
# mc_cv() with strata cannot handle NA values in the strata variable
data_clean <- data %>% filter(!is.na(target))
if (nrow(data_clean) < nrow(data)) {
  warning(sprintf("Removed %d rows with NA target values before MC-CV", nrow(data) - nrow(data_clean)))
}
data <- data_clean

# Also update data_rf to match (remove same rows if they exist)
if (exists("data_rf")) {
  data_rf_clean <- data_rf %>% filter(!is.na(target))
  if (nrow(data_rf_clean) < nrow(data_rf)) {
    warning(sprintf("Removed %d rows from data_rf with NA target values", nrow(data_rf) - nrow(data_rf_clean)))
  }
  data_rf <- data_rf_clean
}

# Validate minimum sample sizes for stratification
target_counts <- table(data$target)
min_samples_needed <- ceiling(N_SPLITS * TEST_SIZE * 2)  # Need at least 2 samples per class per split
cat(sprintf("\nAfter cleaning:\n"))
cat(sprintf("  Total samples: %d\n", nrow(data)))
cat(sprintf("  Target distribution: 0=%d (%.1f%%), 1=%d (%.1f%%)\n", 
            target_counts[1], 100*target_counts[1]/sum(target_counts),
            target_counts[2], 100*target_counts[2]/sum(target_counts)))
cat(sprintf("  Min samples needed per class: %d (for %d splits with %.0f%% test)\n",
            min_samples_needed, N_SPLITS, TEST_SIZE*100))

if (any(target_counts < min_samples_needed)) {
  warning(sprintf("⚠️  Small sample size detected for stratification!\n"))
  warning(sprintf("   Consider: reducing N_SPLITS, increasing TEST_SIZE, or using more data.\n"))
}

mc_splits <- mc_cv(
  data = data,
  prop = TRAIN_PROP,
  times = N_SPLITS,
  strata = target  # Stratified by target (bare name - rsample uses non-standard evaluation)
)

cat(sprintf("\n✓ Created %d MC-CV splits (stratified)\n", N_SPLITS))
cat("Note: CatBoost uses categorical features, Random Forest uses one-hot encoded features\n")


In [ ]:
# Run MC-CV for a single model type
run_mc_cv_method <- function(data, method, mc_splits, data_rf = NULL) {
  cat(sprintf("\n--- Running MC-CV for %s ---\n", method))
  
  # Use appropriate data format for each model
  if (method == "catboost") {
    # CatBoost: use categorical format
    # Check if target column exists
    if (!"target" %in% names(data)) {
      stop("Target column not found in data. Check that data includes target column.")
    }
    X <- data %>% select(-target)
    y <- data$target
    
    # Validate y
    if (all(is.na(y))) {
      stop("Target column (y) is all NA. Check data loading and target extraction.")
    }
    if (DEBUG_MODE) {
      cat(sprintf("  Data check: nrow=%d, ncol=%d, target NAs=%d, target positives=%d\n",
                  nrow(data), ncol(data), sum(is.na(y)), sum(y == 1, na.rm = TRUE)))
    }
  } else if (method == "random_forest") {
    # Random Forest: use one-hot encoded format
    if (is.null(data_rf)) {
      stop("Random Forest requires data_rf (one-hot encoded format)")
    }
    if (!"target" %in% names(data_rf)) {
      stop("Target column not found in data_rf. Check that data_rf includes target column.")
    }
    X <- data_rf %>% select(-target)
    y <- data_rf$target
    
    # Validate y
    if (all(is.na(y))) {
      stop("Target column (y) in data_rf is all NA. Check data loading and target extraction.")
    }
  } else {
    stop(sprintf("Unknown method: %s", method))
  }
  
  feature_names <- colnames(X)
  
  # Create progress bar
  p <- progressor(steps = N_SPLITS)
  
  # Run MC-CV in parallel
  results <- future_map(1:N_SPLITS, function(i) {
    p()
    
    # Get train/test split
    split <- mc_splits$splits[[i]]
    train_idx <- split$in_id
    test_idx <- split$out_id
    
    # Validate indices (handle empty vectors and NA values)
    if (length(train_idx) == 0) {
      stop(sprintf("[Split %d] Empty train_idx", i))
    }
    if (length(test_idx) == 0) {
      stop(sprintf("[Split %d] Empty test_idx", i))
    }
    
    # Check for NA values in indices and remove them
    if (any(is.na(train_idx))) {
      warning(sprintf("[Split %d] train_idx contains %d NA values, removing them", 
                      i, sum(is.na(train_idx))))
      train_idx <- train_idx[!is.na(train_idx)]
    }
    if (any(is.na(test_idx))) {
      warning(sprintf("[Split %d] test_idx contains %d NA values, removing them", 
                      i, sum(is.na(test_idx))))
      test_idx <- test_idx[!is.na(test_idx)]
    }
    
    # Re-check after removing NAs
    if (length(train_idx) == 0) {
      stop(sprintf("[Split %d] train_idx is empty after removing NAs", i))
    }
    if (length(test_idx) == 0) {
      stop(sprintf("[Split %d] test_idx is empty after removing NAs", i))
    }
    
    max_train <- max(train_idx, na.rm = TRUE)
    max_test <- max(test_idx, na.rm = TRUE)
    
    if (is.infinite(max_train) || is.infinite(max_test)) {
      stop(sprintf("[Split %d] Invalid max indices: max(train_idx)=%s, max(test_idx)=%s",
                   i, ifelse(is.infinite(max_train), "Inf", as.character(max_train)),
                   ifelse(is.infinite(max_test), "Inf", as.character(max_test))))
    }
    
    if (max_train > length(y) || max_test > length(y)) {
      stop(sprintf("[Split %d] Index out of bounds: max(train_idx)=%d, max(test_idx)=%d, length(y)=%d",
                   i, max_train, max_test, length(y)))
    }
    
    X_train <- X[train_idx, , drop = FALSE]
    X_test <- X[test_idx, , drop = FALSE]
    y_train <- y[train_idx]
    y_test <- y[test_idx]
    
    # Validate extracted y_test
    if (all(is.na(y_test))) {
      warning(sprintf("[Split %d] y_test is all NA. y length=%d, test_idx length=%d, y NAs=%d",
                      i, length(y), length(test_idx), sum(is.na(y))))
    }
    
    # Train model
    model <- NULL
    if (method == "catboost") {
      model <- train_catboost_r(X_train, y_train, MODEL_PARAMS$catboost)
    } else if (method == "random_forest") {
      model <- train_random_forest_r(X_train, y_train, MODEL_PARAMS$random_forest)
    } else {
      stop(sprintf("Unknown method: %s", method))
    }
    
    # Get predictions (binary and probabilities)
    if (method == "catboost") {
      y_pred <- predict_catboost_r(model, X_test)
      y_pred_proba <- predict_proba_catboost_r(model, X_test)
    } else if (method == "random_forest") {
      y_pred <- predict_random_forest_r(model, X_test)
      y_pred_proba <- predict_proba_random_forest_r(model, X_test)
    }
    
    # Calculate metrics based on SCALING_METRIC setting
    if (SCALING_METRIC == "recall") {
      # Calculate Recall
      if (DEBUG_MODE && i <= 2) {
        cat(sprintf("  [Split %d] y_test: length=%d, NAs=%d, positives=%d\n", 
                    i, length(y_test), sum(is.na(y_test)), sum(y_test == 1, na.rm = TRUE)))
        cat(sprintf("  [Split %d] y_pred: length=%d, NAs=%d, positives=%d\n", 
                    i, length(y_pred), sum(is.na(y_pred)), sum(y_pred == 1, na.rm = TRUE)))
      }
      metric_value <- calculate_recall(y_test, y_pred)
    } else if (SCALING_METRIC == "logloss") {
      # Calculate LogLoss (inverted for scaling: 1/logloss so higher = better)
      logloss <- calculate_logloss(y_test, y_pred_proba)
      metric_value <- if (is.infinite(logloss) || logloss == 0) 0 else (1 / logloss)
      if (DEBUG_MODE && i <= 2) {
        cat(sprintf("  [Split %d] LogLoss: %.4f, Inverted: %.4f\n", i, logloss, metric_value))
      }
    } else {
      stop(sprintf("Unknown SCALING_METRIC: %s. Use 'recall' or 'logloss'", SCALING_METRIC))
    }
    
    # Get feature importance
    if (method == "catboost") {
      # CatBoost handles character columns directly - no conversion needed
      importance <- get_importance_catboost_r(model, X_test)
    } else if (method == "random_forest") {
      importance <- get_importance_random_forest_r(model)
    }
    
    # Validate importance vector
    expected_length <- if (method == "catboost") ncol(X) else ncol(X_rf)
    if (length(importance) != expected_length) {
      warning(sprintf("[Split %d] Importance length mismatch: got %d, expected %d. Using zeros.", 
                      i, length(importance), expected_length))
      importance <- setNames(rep(0, expected_length), 
                            if (method == "catboost") names(X) else names(X_rf))
    }
    
    # Return results
    list(
      model = model,
      metric_value = metric_value,
      importance = importance
    )
  }, .options = furrr_options(seed = 42))
  
  # Extract results
  metric_values <- map_dbl(results, ~ .x$metric_value)
  importance_list <- map(results, ~ .x$importance)
  
  # Validate importance vectors before creating matrix
  importance_lengths <- map_int(importance_list, length)
  expected_length <- length(feature_names)
  
  if (any(importance_lengths != expected_length)) {
    warning(sprintf("Importance length mismatch detected. Expected %d, got: %s", 
                    expected_length, paste(unique(importance_lengths), collapse = ", ")))
    # Fix any mismatched importance vectors
    importance_list <- map(importance_list, function(imp) {
      if (length(imp) != expected_length) {
        if (length(imp) == 1) {
          # Single value - create zero vector
          setNames(rep(0, expected_length), feature_names)
        } else {
          # Wrong length - pad or truncate
          if (length(imp) < expected_length) {
            imp <- c(imp, rep(0, expected_length - length(imp)))
          } else {
            imp <- imp[1:expected_length]
          }
          setNames(imp, feature_names)
        }
      } else {
        imp
      }
    })
  }
  
  # Create matrix from validated importance vectors
  importance_matrix <- do.call(rbind, importance_list)
  
  # Validate matrix dimensions
  if (ncol(importance_matrix) != expected_length) {
    stop(sprintf("Importance matrix has wrong number of columns: got %d, expected %d", 
                 ncol(importance_matrix), expected_length))
  }
  
  # Average importance across splits
  avg_importance <- colMeans(importance_matrix)
  names(avg_importance) <- feature_names
  
  # Normalize importance (0-1 scale)
  min_imp <- min(avg_importance)
  max_imp <- max(avg_importance)
  if (max_imp > min_imp) {
    normalized_importance <- (avg_importance - min_imp) / (max_imp - min_imp)
  } else {
    normalized_importance <- rep(1 / length(avg_importance), length(avg_importance))
  }
  
  # Scale by mean MC-CV metric value
  mean_metric <- mean(metric_values)
  scaled_importance <- normalized_importance * mean_metric
  
  # Create results DataFrame
  metric_name <- if (SCALING_METRIC == "recall") "recall" else "logloss_inverted"
  results_df <- tibble(
    feature = feature_names,
    importance_raw = avg_importance,
    importance_normalized = normalized_importance,
    importance_scaled = scaled_importance,
    model_type = method,
    mc_cv_metric_mean = mean_metric,
    mc_cv_metric_std = sd(metric_values)
  ) %>%
    arrange(desc(importance_scaled)) %>%
    mutate(rank = row_number())
  
  # Rename metric column for clarity
  names(results_df)[names(results_df) == "mc_cv_metric_mean"] <- sprintf("mc_cv_%s_mean", metric_name)
  names(results_df)[names(results_df) == "mc_cv_metric_std"] <- sprintf("mc_cv_%s_std", metric_name)
  
  cat(sprintf("  Mean %s: %.4f ± %.4f\n", 
              if (SCALING_METRIC == "recall") "Recall" else "LogLoss (inverted)", 
              mean_metric, sd(metric_values)))
  cat(sprintf("  Top 50 features:\n"))
  top50 <- head(results_df, 50)
  for (i in 1:nrow(top50)) {
    cat(sprintf("    %d. %s (scaled=%.6f)\n", 
                top50$rank[i], top50$feature[i], top50$importance_scaled[i]))
  }
  
  return(results_df)
}

cat("✓ MC-CV function defined\n")


## 6. Run Analysis

Run MC-CV for each model type.


In [ ]:
# Run each method
methods <- c("catboost", "random_forest")
all_results <- list()

# Note: data_rf was created in the feature engineering step above
# Both data and data_rf should have the same number of rows and same patient order

for (method in methods) {
  if (method == "random_forest") {
    result <- run_mc_cv_method(data, method, mc_splits, data_rf = data_rf)
  } else {
    result <- run_mc_cv_method(data, method, mc_splits)
  }
  all_results[[method]] <- result
  
  # Save individual results
  output_file <- file.path(output_dir, sprintf("%s_%s_%s_%d_%s_feature_importance.csv",
                                                COHORT_NAME, AGE_BAND, EVENT_YEAR, method))
  write_csv(result, output_file)
  cat(sprintf("Saved: %s\n", output_file))
}


## 7. Aggregate Results

Combine results across models.


In [ ]:
# Aggregate across models
cat("\n========================================\n")
cat("Aggregating Results Across Models\n")
cat("========================================\n")

combined_df <- bind_rows(all_results)

# Aggregate by feature (average scaled importance)
# Get metric column names dynamically
metric_mean_col <- sprintf("mc_cv_%s_mean", if (SCALING_METRIC == "recall") "recall" else "logloss_inverted")
metric_std_col <- sprintf("mc_cv_%s_std", if (SCALING_METRIC == "recall") "recall" else "logloss_inverted")

aggregated <- combined_df %>%
  group_by(feature) %>%
  summarise(
    importance_raw = mean(importance_raw),
    importance_normalized = mean(importance_normalized),
    importance_scaled = mean(importance_scaled),
    metric_mean = mean(.data[[metric_mean_col]]),
    metric_std = mean(.data[[metric_std_col]]),
    .groups = 'drop'
  ) %>%
  arrange(desc(importance_scaled)) %>%
  mutate(rank = row_number())

# Rename metric columns for clarity
names(aggregated)[names(aggregated) == "metric_mean"] <- metric_mean_col
names(aggregated)[names(aggregated) == "metric_std"] <- metric_std_col

# Save aggregated results
output_file <- file.path(output_dir, sprintf("%s_%s_%d_feature_importance_aggregated.csv",
                                              COHORT_NAME, AGE_BAND, EVENT_YEAR))
write_csv(aggregated, output_file)
cat(sprintf("Saved: %s\n", output_file))

# Print summary
cat("\n========================================\n")
cat("Summary\n")
cat("========================================\n")
cat(sprintf("Total features: %d\n", nrow(aggregated)))
cat(sprintf("Models used: %s\n", paste(methods, collapse = ", ")))
cat(sprintf("Mean MC-CV %s: %.4f\n", 
            if (SCALING_METRIC == "recall") "Recall" else "LogLoss (inverted)",
            mean(aggregated[[metric_mean_col]])))
cat("\nTop 50 features:\n")
top50 <- head(aggregated, 50)
for (i in 1:nrow(top50)) {
  cat(sprintf("  %2d. %-40s | scaled=%.6f | %s=%.4f\n",
              top50$rank[i], top50$feature[i], top50$importance_scaled[i], 
              if (SCALING_METRIC == "recall") "recall" else "logloss_inv",
              top50[[metric_mean_col]][i]))
}


# Cleanup

Close parallel processing.


In [ ]:
# Close parallel processing
plan(sequential)

cat("\n========================================\n")
cat("Analysis Complete!\n")
cat("========================================\n")
cat(sprintf("Output directory: %s\n", output_dir))
cat(sprintf("MC-CV splits: %d\n", N_SPLITS))
cat(sprintf("Train/Test ratio: %.0f/%.0f\n", TRAIN_PROP * 100, TEST_SIZE * 100))
cat("\nResults show scaled feature importance with MC-CV Recall scores\n")
cat("based on", N_SPLITS, "independent train/test splits.\n")


# Sync Results and Code to S3

Sync output files and code (notebook + R script) to S3 bucket. 
- Outputs: CSV results files
- Code: Notebook and R script for reproducibility

In [ ]:
# Sync outputs and code to S3
# On EC2, we're in the feature_importance directory
s3_bucket <- "s3://pgx-repository/pgx-analysis/3_feature_importance/"

# Find AWS CLI (check common locations - EC2 typically has it in /usr/local/bin or /usr/bin)
aws_cmd <- Sys.which("aws")
if (aws_cmd == "") {
  # Try common EC2 installation paths
  aws_paths <- c(
    "/usr/local/bin/aws",
    "/usr/bin/aws",
    "/home/ec2-user/.local/bin/aws"
  )
  aws_cmd <- NULL
  for (path in aws_paths) {
    if (file.exists(path)) {
      aws_cmd <- path
      break
    }
  }
  if (is.null(aws_cmd)) {
    stop("AWS CLI not found. Please install AWS CLI or ensure it's in your PATH.")
  }
}

cat("Syncing outputs and code to S3...\n")
cat("Source: feature_importance/ directory\n")
cat("Destination:", s3_bucket, "\n")
cat("AWS CLI:", aws_cmd, "\n\n")

# Get current directory (should be feature_importance)
current_dir <- getwd()
if (!grepl("feature_importance", current_dir)) {
  warning("Current directory doesn't appear to be feature_importance. Double-check sync destination.")
}

# Sync feature_importance directory (includes outputs/ and code files)
# Explicitly include notebook, R scripts, README files, and outputs directory
# Exclude temporary files, checkpoints, and unnecessary directories
# Note: --delete flag removed for safety (won't delete files in S3 that don't exist locally)
# Include patterns are processed before exclude patterns, then exclude everything else
sync_cmd <- sprintf(
  '"%s" s3 sync "%s" %s --include "*.ipynb" --include "*.R" --include "README*.md" --include "outputs/**" --exclude "*checkpoint*" --exclude "*.tmp" --exclude "*.ipynb_checkpoints/*" --exclude "*.RData" --exclude "*.Rhistory" --exclude ".Rproj.user/*" --exclude "catboost_info/*" --exclude "*.log" --exclude "*"',
  aws_cmd,
  current_dir,
  s3_bucket
)

cat("Running:", sync_cmd, "\n\n")
result <- system(sync_cmd)

if (result == 0) {
  cat("✓ Successfully synced outputs and code to S3\n")
  cat("  - Outputs:", file.path(output_dir), "\n")
  cat("  - Code: *.ipynb, *.R, README*.md\n")
} else {
  warning(sprintf("S3 sync returned exit code %d. Check AWS credentials and permissions.", result))
}


# Shutdown EC2

In [ ]:

# Shutdown EC2 instance after analysis completes
# Set SHUTDOWN_EC2 = TRUE to enable, FALSE to disable
SHUTDOWN_EC2 <- TRUE  # Change to TRUE to enable auto-shutdown

if (SHUTDOWN_EC2) {
  cat("\n========================================\n")
  cat("Shutting down EC2 instance...\n")
  cat("========================================\n")
  
  # Get instance ID from EC2 metadata service
  instance_id <- tryCatch({
    system("curl -s http://169.254.169.254/latest/meta-data/instance-id", intern = TRUE)
  }, error = function(e) {
    cat("Warning: Could not retrieve instance ID from metadata service.\n")
    cat("If running on EC2, check that metadata service is accessible.\n")
    return(NULL)
  })
  
  if (!is.null(instance_id) && length(instance_id) > 0 && nchar(instance_id[1]) > 0) {
    instance_id <- instance_id[1]
    cat(sprintf("Instance ID: %s\n", instance_id))
    
    # Find AWS CLI
    aws_cmd <- Sys.which("aws")
    if (aws_cmd == "") {
      aws_paths <- c(
        "/usr/local/bin/aws",
        "/usr/bin/aws",
        "/home/ec2-user/.local/bin/aws"
      )
      aws_cmd <- NULL
      for (path in aws_paths) {
        if (file.exists(path)) {
          aws_cmd <- path
          break
        }
      }
    }
    
    if (!is.null(aws_cmd) && aws_cmd != "") {
      # Stop the instance (use terminate-instances for permanent deletion)
      shutdown_cmd <- sprintf(
        '"%s" ec2 stop-instances --instance-ids %s',
        aws_cmd,
        instance_id
      )
      
      cat("Running:", shutdown_cmd, "\n")
      result <- system(shutdown_cmd)
      
      if (result == 0) {
        cat("✓ EC2 instance stop command sent successfully\n")
        cat("Instance will stop in a few moments.\n")
        cat("Note: This is a STOP (not terminate), so you can restart it later.\n")
      } else {
        warning(sprintf("EC2 stop command returned exit code %d. Check AWS credentials and permissions.", result))
      }
    } else {
      cat("Warning: AWS CLI not found. Cannot shutdown instance.\n")
      cat("Install AWS CLI or ensure it's in your PATH.\n")
    }
  } else {
    cat("Warning: Could not determine instance ID. Skipping shutdown.\n")
    cat("If you want to shutdown manually, use:\n")
    cat("  aws ec2 stop-instances --instance-ids <your-instance-id>\n")
  }
} else {
  cat("\n========================================\n")
  cat("EC2 Auto-Shutdown: DISABLED\n")
  cat("========================================\n")
  cat("To enable auto-shutdown, set SHUTDOWN_EC2 = TRUE in this cell.\n")
  cat("Instance will continue running.\n")
}
